[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/CCS-ZCU/pribehy-dat/blob/master/scripts/gis.ipynb)

# GIS: Analýza prostorových dat

**autor**: *Vojtěch Kaše* (kase@ff.zcu.cz)

[![](https://ccs.zcu.cz/wp-content/uploads/2021/10/cropped-ccs-logo_black_space_240x240.png)](https://ccs.zcu.cz)

## Úvod a cíle kapitoly

V této kapitole si ukážeme základy analytické práce s prostorovými daty, tzv. GIS (=Geographic Information System). Budeme pracovat výlučně s tzv. vektorovým modelem dat, v rámci kterého jsou body, linie a plochy na mapě definovány výčtem bodů definujících zeměpisné šířky a délky. Body obsahují pouze jeden pár souřadnic (např. místo, kde se nachází strom); linie obsahují řadu párů souřadnic (např. kudy vede silnice) a plochy obsahují řadu párů souřadnic, které ohraničují určité území (např. stát).  Body, linie a plochy mohou být dále obohaceny o další atributy. 

Cvičení bude postaveno na veřejně dostupných datesetech antických nápisů v latině a řečtině. 

V rámci Pythonu pro nás bude stěžejní knihovna `geopandas`, která představuje rozšíření knihovny pandas pro práci s prostorovými daty. Základním typem dat bude tzv. `GeoDataFrame`. GeoDataFrame se chová v zásadě stejně jako DataFrame, s tím rozdílem, že obsahuje sloupec navíc, s názvem "geometry", který definuje právě zeměpisné souřadnice. Tato vlastnost umožňuje tabulková data v tomto formátu bezprostředně vykreslit do mapy.

## Cvičení

In [ ]:
%%capture

# nyní si naimportujeme některé rozšiřující knihovny, které budeme níže používat (metaforicky řečeno si nasadíme nástavce...)
#!pip install mapclassify
import requests
import shutil
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as colors
mpl.rcParams["axes.edgecolor"] = "white"
mpl.rcParams["xtick.color"] = "white"
mpl.rcParams["xtick.labelcolor"] = "white"
mpl.rcParams["ytick.color"] = "white"
mpl.rcParams["ytick.labelcolor"] = "white"
pd.set_option('display.max_columns', 100)

In [ ]:
# v této buňce si náš virtuální stroj stáhneme první důležitý dataset
# dataset má přes 100MB, tudíž to pár vteřin potrvá
!curl https://zenodo.org/records/10146150/files/LIST_v1-1.parquet --output LIST_v1-1.parquet

In [ ]:
# dataset si načteme do naší aktuální "session"
LIST = gpd.read_parquet("LIST_v1-1.parquet")

Když máme dataset načtený, můžeme s ním dále pracovat - aplikovat na něj různé funkce/metody a dále jej upravovat, filtrovat či rozšiřovat o další atributy
Podívejme se nyní v krátkosti, jak tento dataset vypadá. Oomocí metody `shape` zjístím, jaký má tabulka tvar:



In [ ]:
LIST.shape

První z dvojice čísel udává počet řádků tabulky a druhá počet sloupců tabulky.

další užitečné informace získáme, necháme-li si vypsat prvních 5 řádků pomocí metody `head()`

In [ ]:
LIST.head()

In [ ]:
# pro přehlednost si můžeme nechat vypsat kompletní seznam sloupců:
LIST.columns

pro naše účely jsou mnohé atributy (sloupce) zcela nepotřebné. Pro další analýzy se proto omezíme pouze na několik z nich, a to "LIST_ID", "is_geotemporal", "urban_context", "not_before", "not_after", "Longitude", "Latitude" a "geometry".

In [ ]:
# pomocí dvojitých hranatých závorek vyfiltrujme z datasetu pouze vybrané sloupce
LIST = LIST[["LIST-ID", "Longitude", "Latitude", "geometry",  "not_before", "not_after", "is_geotemporal"]]
# podívejme se na náhodný vzorek 10 nápisů:
LIST.sample(10, random_state=0)

Atribut ve sloupci "LIST-ID" obsahuje jednoznačný číselný identifikátor daného nápisu v našem datasetu.

Atributy "Longitude" a "Latitude" vyjadřují zeměpisnou délku a šířku.

Stejná informace je i ve sloupci "geometry". To však není jen tak ledajaký sloupec! Je to sloupec, který z našich dat dělá prostorová data zpracovatelná knihovnou `geopandas`. Data ve sloupci "geometry"  tvoří tzv. [`GeoSeries`](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoSeries.html#geopandas-geoseries) a celá tabulka je díky nim takzvaný [`GeoDataFrame`](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.html). Hodnoty zeměpisné šířky a délky jsou zde specificky naformátované a samotné číselné hodnoty předchází slovo "POINT". Definuje se zde, že se jedná o bodové geometrie (formátování jednotlivých prvků vychází ze standardů  python knihovny [Shapely](https://shapely.readthedocs.io/en/stable/manual.html)). Díky těmto vlastnostem lze daná tabulková data bezprostředně použít k nejrůznějším prostorovým operacím, z nichž některé si za okamžik ukážeme.

Sloupce `not_before` a `not_after` vyjadřují časovou informaci. Definují, ze kdy daný nápis pochází. S historickými daty jsme často jako badatelé v situaci, kdy tuto dataci neznáme přesně, jsme schopni pouze schopni vyjádřit informovaný odhad ve formě intervalu. V případě antiky jsme velké množství objektů schopni datovat pouze do konkrétního století (např. 4. stol.n.l.). To lze pak číselně vyjádřit intervalem 301-400, jako je tomu v případě posledního nápisu v našem vzorku. V jiných případech daný objekt obsahuje indície, které umožňují mnohem přesnější dataci (například tím, že odkazují k nějaké známé historické události). V takovém případě je hodnota `not_before` a `not_after` totožná.

Sloupec "is_geotemporal" obsahuje boolovské hodnoty *Pravda* (True) nebo *Nepravda* (False) odvozené z hodnot ve sloupcích "geometry", "not_before" a "not_after". Hodnota True je zde tehdy, pokud má nápis jak validní nenulovou geometrie a tak informaci o dataci, což v našem datasetu není vždy tak.

Tento sloupec  nyní využijeme k dodatečnému filtrování našich dat, kdy se omezíme pouze na položku, kdy tento atribut má hodnotu True.

In [ ]:
LIST = LIST[LIST["is_geotemporal"]]
LIST.shape

opět jsme použili metodu shape, abychom se podívali, jaký má náš dataset nyní "tvar". Vidíme, že po filtrování sestává pouze z cca. dvou set tisíc položek a osmi sloupců.

Další důležitá vlastnost našeho `GeoDataFrame` objektu se skrývá za atributem `crs`. Tato tři písmena představují zkratku. Věděli bychom jakou? Zkusme se podívat na výsledek:

In [ ]:
LIST.crs

Nyní již můžeme postoupit k první prostorové vizualizaci:

In [ ]:
LIST.plot(color="black", markersize=1)

Výsledek není nijak atraktivní a sám o sobě asi i těžko interpretovatelný. V podstatě jsme jen vykreslili do prostoru o něco málo více než 200 000 tisíc bodů. V dalších krocích tohoto workshopu však dostatneme tuto vizualizaci do vizuálně přitažlivější podoby...

V následujícím kroku si načteme další dataset. Tentokrát přímo z webového odkazu:

In [ ]:
provinces = gpd.read_file("https://raw.githubusercontent.com/sdam-au/GI_ETL/master/data/provinces_valid.geojson")

In [ ]:
# opět si nejprve prohlédneme pět prvních řádek atributové tabulky
provinces.head()

In [ ]:
provinces.shape

In [ ]:
provinces.crs

Tento dataset má pouze dva sloupce: "province" a opět "geometry". Vidíme však, že sloupec "geometry" vypadá nyní odlišně. Na první řádce čteme "MULTIPOLYGON" a na dalších řádcích "POLYGON". Naše geometrie zde tedy již nesestávají z bodů ale z ploch. Jak název napovídá, jedná se o provincie, a totiž o provincie Římské říše, její hlavní správní celky, obdoba např. českých krajů (viz [Římské provincie](https://cs.wikipedia.org/wiki/%C5%98%C3%ADmsk%C3%A9_provincie)).

I tento dataset si můžeme velice přímočaře vizualizovat pomocí metody `plot()`, nejprve samostatně a následně společně s naším datasetem nápisů jakožto vrchní vrstvou:

In [ ]:
provinces.plot(color="lightgrey", edgecolor="black")

In [ ]:
point_color = "darkred"
ax = provinces.plot(color="grey")
LIST.plot(markersize=1, ax=ax, color=point_color)

In [ ]:
# Tato buňka slouží ke kontrole průchodu tímto cvičením. 
# Pokud toto cvičení plníte v rámci svých studijních povinností na ZČU, buňku spusťte a držte se instrukcí.
exec(requests.get("https://sciencedata.dk/shared/856b0a7402aa7c7258186a8bdb329bd3?download").text)
kontrola_pruchodu(ntb="gis", arg1=point_color)

Na obrázku výše vidíme, že oba datasety se týkají téhož území, avšak že bodu v ploše nejsou zdaleka rovnoměrně rozprostřeny a mnohé se nacházejí zcela vně polygonů provincií. To si zaslouží bližší ohledání.

Nejprve pro každý územní celek (provincii) sečteme počet nápisů, které se v ní nacházejí. Využijeme metody `contains()` (určení pro všechny prvky z datasetu nápisů, zda sa nacházejí či nenacházejí uvnitř onoho území (True nebo False)).a `sum()` (sečtení všech prvků s hodnotou True.


In [ ]:
%%time
provinces["LIST_n"] = provinces.geometry.apply(lambda x: x.contains(LIST.geometry).sum())
provinces.head(5)

v datasetu provincií nyní máme informaci o tom, kolik se uvnitř ní nachází nápisů. Provincie jsou ale různě velké, a tudíž samotné číslo nemá příliš velkou výpovědní hodnotu. Spočítejme tedy nyní plochu jednotlivých provincií a vytvořme si nový atribut s názvem "area". Pro tento výpočet si pomocí metody `to_crs` převedeme do metrické projekce EPSG:3035 a získáme plochu v metrech čtverečních. Kilomtery čtvereční získáme, když výsledné hodnoty vydělíme miliónem.

In [ ]:
provinces["area_km2"] = provinces.to_crs(3035).area / 1000000
provinces.head(5)

Pro každou provincii tedy nyní máme a) množtví latinských nápisů, které z ní pocházejí a b) rozsah jejího území v kilometrech čtverečních. Jak získáme hustotu?

In [ ]:
provinces["LIST_density"] = provinces["LIST_n"] / provinces["area_km2"]

Data si prohlédneme uspořádaná od položek s největší hodnotou tohoto nového atributu:

In [ ]:
provinces.sort_values("LIST_density", ascending=False, inplace=True)
provinces.head(5)

Vydíme zde výraznou vyjímečnost dat z města Říma (první řádek) a jeho nejbližšího okolí (Lacia, první polygon je současně obsažen v druhém), kde máme řádově vyšší hodnoty, než kdekoli jinde. Proto je z následující vizualizace vyloučíme.

Danou hodnotu vyjádříme prostřednictvím baravného odstínu příslušné provincie:

In [ ]:
provinces.plot(column=provinces["LIST_density"], cmap="Reds", edgecolor="black", scheme="Quantiles", k=20)

Vidíme, že latinské nápisy se akumulují převážně v západní části Římské říše. To je i území, o němž víme, že na něm dominovala latina. Zatímco ve východní části říše dominovala řečtina.

Proto si nyní stáhneme a načteme ještě jeden dataset. Opět se jedná o dataset nápisů, tentokrát však v řečtině.


In [ ]:
# nyní si stáhneme a načteme ještě jeden dataset. Opět se jedná o dataset nápisů, tentokrát však v řečtině-
!curl https://zenodo.org/records/10139110/files/GIST_v1-1.parquet --output GIST_v1-1.parquet

In [ ]:
GIST = gpd.read_parquet("GIST_v1-1.parquet")
GIST.shape

Opět se omezíme pouze na vybrané atributy, které budou s výjimkou první (identifikátor) totožné jako v předchozím případě: "PHI_ID", "Longitude", "Latitude", "geometry",  "not_before", "not_after" a  "is_geotemporal". Současně rovnou provedeme filtraci a omezíme se pouze na ty s validní geometrií a datací.


In [ ]:
GIST = GIST[["PHI_ID", "Longitude", "Latitude", "geometry", "not_before", "not_after", "is_geotemporal"]]
GIST = GIST[GIST["is_geotemporal"]]
GIST.sample(10, random_state=0)

Následně si můžeme opět minimalisticky vizualizovat všechny tyto nápisy společně s našimi dalšími dvěma datasety:

In [ ]:
ax = provinces.plot(color="lightgrey")
LIST.plot(markersize=1, ax=ax, color="darkred", alpha=0.5)
GIST.plot(markersize=1, ax=ax, color="darkblue", alpha=0.5)

Vidíme zde poměrně jasný prostorový vzor. Povšimneme, že řecké nápisy dominují od dnešního Chorvatska směrem dále na východ. Pojďme ale toto pozorování rozpracovat numericky. Spočtěme si počty řeckých nápisů v jednotlivých provinciích a též jejich hustoty.

# Samostatný úkol: počet a hustota řeckých nápisů uvnitř jednotlivých provincií:

Povšimli jsme si, že dataset řeckých nápisů `GIST` má obdobnou strukturu jako dataset latinských nápisů `LIST`.

V případě latinských nápisů jsme výše také spočítali jejich počet uvnitř jednotlivých provincií v datasetu `provinces`, když jsme zde aplikovali funkce `contains()` a `sum()`. Dataset `provinces` jsme tak obohatili o nový atribut s názvem `"LIST_n"`. Dále jsme vytvořili atribut `"area_sk"` a nakonec - za využití těchto dvou předchozích atributů - atribut `"LIST_density"`.

Vašim **samostatným úkolem** nyní je obohatit dataset `provinces` o dva další atributy, které budou obsahovat informace o počtech a hustotě řeckých nápisů z datasetu `GIST`. Nazvete je `"GIST_n"` a "`GIST_density`".

(Nápověda: V zásadě jde o to buněk níže zkopírovat a mírně poupravit několik řádek kódu, které jsme použili výše k vytvoření atributů `"LIST_n"` a `"LIST_density"`.)



In [ ]:
# ÚKOL 1
# přidejme k datasetu provinces atribut "GIST_n" (za využití kombinace funkcí apply(), contains() a sum() - viz výše v případě latinských nápisů)



In [ ]:
# výsledek si prohlédneme:
provinces.head(5)

In [ ]:
# ÚKOL 2:
# přidejme k datasetu provinces atribut "GIST_density" (počty nápisů v atributu "GIST_n" vydělíme hustutou v atributu "area_sk" - viz obdobný výpočet výše v případě latinských nápisů)



Pokud jsme postupovali správně, půjdou nám spustit i buňky kódu níže.

In [ ]:
fig, ax1 = plt.subplots()
provinces.plot(column=provinces["GIST_density"], cmap="Blues", edgecolor="black", scheme="Quantiles", k=20, ax=ax1)

# tentokrát si vytvoříme i legendu
# budeme postupovat krok po kroku, abychom měli plnou kontrolu
mpl.rcParams["ytick.labelcolor"] = "black"
vmin = np.round(provinces["GIST_density"].min(), 2)
vmax = np.round(provinces["GIST_density"].max(), 2)
ax1.set_xticklabels([])
ax1.set_yticklabels([])

sm = plt.cm.ScalarMappable(cmap='Blues', norm=plt.Normalize(vmin=vmin, vmax=vmax))
cbar = plt.colorbar(sm, ax=ax1, orientation='horizontal', shrink=0.5)
cbar.set_ticks([vmin, vmax])
cbar.set_label('GIST density (inscriptions per km$^2$)')

Nyní zkombinujeme oba datasety dohromady

In [ ]:
# součet řeckých a latinských nápisů
provinces["GLIST_n"] = provinces["LIST_n"] + provinces["GIST_n"]

In [ ]:
# hustota součtu řeckých a latinských nápisů
provinces["GLIST_density"] = provinces["GLIST_n"] / provinces["area_km2"]

In [ ]:
# proporcionální zastoupení latinských nápisů na škále od jedné do nuly (1=pouze latinské, 0=pouze řecké)
provinces["LIST_proportion"] = provinces["LIST_n"] / provinces["GLIST_n"]

In [ ]:
fig, ax1 = plt.subplots()
provinces.plot(column=provinces["LIST_proportion"], cmap="seismic", edgecolor="black", ax=ax1)
ax1.set_yticklabels([])
ax1.set_xticklabels([])

mpl.rcParams["xtick.labelcolor"] = "black"
sm = plt.cm.ScalarMappable(cmap='seismic') #, norm=plt.Normalize(vmin=0, vmax=1))
cbar = plt.colorbar(sm, ax=ax1, orientation='horizontal', shrink=0.5)
cbar.set_ticks([0, 0.5, 1])
cbar.set_label('LIST proportion (0=no latin; 1=latin only)')

Tento obrázek si nyní uložíme.

In [ ]:
fig.savefig("provinces_list-proportion.png")
#files.download("provinces_list-proportion.png")

# Rozšiřující analýza: časový rozměr

In [ ]:
%%capture
# doinstalujeme si knihovnu pro práci s "časovou nejistotou" v historických datech: https://pypi.org/project/tempun/
!pip install tempun
import tempun

In [ ]:
# datace jednotlivých nápisů jsou v tuto chvíli vyjádřeny interval, s touto formou dat se rozložení dat v čase špatně analyzuje
# datační interval využijeme nyní k tomu, že si každý nápis datujeme do jednoho konkrétního roku ohraničeného tímto intervalem
# POZOR: provedení tohoto příkazu trvá několik minut
def get_one_random_year(row):
  year = tempun.model_date(row["not_before"], row["not_after"], size=2)[0]
  return year
GIST["year"] = GIST.apply(lambda row: get_one_random_year(row), axis=1)
LIST["year"] = LIST.apply(lambda row: get_one_random_year(row), axis=1)

In [ ]:
# vytvoříme histogram distribuce nápisů v čase
fig, ax = plt.subplots()
GIST["year"].hist(color="darkred", bins=range(-800,600,50), histtype='step', ax=ax)
LIST["year"].hist(color="darkblue", bins=range(-800,600,50), histtype='step', ax=ax)
ax.set_ylabel("N of inscriptions")
ax.set_xlabel("year")

In [ ]:
# vytvoříme mapu všech nápisů datovaných před naším letopočtem (nahoře) a našeho letopočtu (dole)
fig, [ax1,ax2] = plt.subplots(2,1, dpi=300, tight_layout=True)

provinces.plot(color="lightgrey", ax=ax1)
GIST[GIST["year"]<=0].plot(color="darkblue", alpha=0.5, markersize=1, ax=ax1)
LIST[LIST["year"]<=0].plot(color="darkred", alpha=0.5, markersize=1, ax=ax1)
ax1.set_xlim(-10,50)
ax1.set_yticklabels([])
ax1.set_xticklabels([])
ax1.set_title("All inscriptions BCE")

provinces.plot(color="lightgrey", ax=ax2)
GIST[GIST["year"]>0].plot(color="darkblue", markersize=1,alpha=0.5, ax=ax2)
LIST[LIST["year"]>0].plot(color="darkred", markersize=1, alpha=0.5, ax=ax2)
ax2.set_xlim(-10,50)
ax2.set_yticklabels([])
ax2.set_xticklabels([])
ax2.set_title("All inscriptions CE")


In [ ]:
# fig.savefig("GIST+LIST_BCvsCE.png")